In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.metrics import categorical_accuracy

Using TensorFlow backend.


In [2]:
VAL_LABELS_FILE = "/home/katarzyna/Desktop/DEEP_LEARNING/datasets/faces/faces_is/val/labels.txt"
TRAIN_LABELS_FILE = "/home/katarzyna/Desktop/DEEP_LEARNING/datasets/faces/faces_is/train/labels.txt"
TEST_LABELS_FILE = "/home/katarzyna/Desktop/DEEP_LEARNING/datasets/faces/faces_is/test/labels.txt"

In [3]:
def get_generator(filename, number=None):
    # załadowanie pliku txt do ramki pandas
    df = pd.read_csv(filename, delimiter=' ', names=["path", "value"], dtype="str", engine='python')
    if number:
        df = df[:number]
    # stworzenie generatora
    gen = image.ImageDataGenerator()
    # folder z danymi, do którego będą doklejane ścieżki z odpowiedniej kolumny
    directory = os.path.dirname(filename)
    # stworzenie iteratora po danych z zadanymi opcjami
    return gen.flow_from_dataframe(df, directory, "path", "value", target_size=(128, 128), batch_size=10)

In [4]:
def show_dataset(flow):
    for img, l in flow:
        # wypisz etykietę
        print(l[0])
        # konwersja obrazu do wyświetlenia
        img = img[0].astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        # wyświetl obraz
        cv2.imshow("image", img)
        # poczekaj na klawisz
        cv2.waitKey()

In [5]:
def create_model(gen):
    model = Sequential()
    model.add(Conv2D(16, (3, 3),input_shape=(128, 128, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(len(gen.class_indices),  activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='sgd',
                 metrics=['categorical_accuracy'])
    return model

In [6]:
def test_model(model):
    test_gen = get_generator(TEST_LABELS_FILE)
    results = model.evaluate_generator(test_gen)
    for name, result in zip(model.metrics_names, results):
        print(f"{name}: {result}")

In [7]:
def get_callbacks():
    callbacks = []
    mc = ModelCheckpoint("weights.hdf5", monitor="val_loss", save_best_only=True, verbose=1)
    es = EarlyStopping(patience=2)
    tb = TensorBoard()
    callbacks.append(mc)
    callbacks.append(es)
    callbacks.append(tb)
    return callbacks

In [8]:
def main():
    train_gen = get_generator(TRAIN_LABELS_FILE, 200)
    val_gen = get_generator(VAL_LABELS_FILE)
    callbacks = get_callbacks()
    model = create_model(train_gen)
    model.load_weights("weights.hdf5")
    #model.fit_generator(train_gen, epochs=100, steps_per_epoch=train_gen.n // train_gen.batch_size, 
    #                    validation_data=val_gen, validation_steps=val_gen.n // val_gen.batch_size,
    #                   callbacks=callbacks)
    test_model(model)

In [9]:
main()

Found 200 validated image filenames belonging to 52 classes.
Found 520 validated image filenames belonging to 52 classes.








Found 572 validated image filenames belonging to 52 classes.
loss: 3.141001524625125
categorical_accuracy: 0.7989510426988135
